### **Deployed Model Link** : https://task-3.streamlit.app/

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import pickle
import warnings
warnings.filterwarnings('ignore')

# Step 1: Load and Preprocess Data
class TitanicData:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = None
        self.X = None
        self.y = None

    def load_data(self):
        self.data = pd.read_csv(self.file_path)
        return self.data

    def preprocess_data(self):
        # Fill missing values
        self.data['Age'].fillna(self.data['Age'].mean(), inplace=True)
        self.data['Embarked'].fillna(self.data['Embarked'].mode()[0], inplace=True)
        self.data['Fare'].fillna(self.data['Fare'].mean(), inplace=True)

        # Drop irrelevant columns
        self.data.drop(columns=['Cabin', 'Ticket', 'Name', 'PassengerId'], inplace=True)

        # Encode categorical variables
        self.data = pd.get_dummies(self.data, columns=['Sex', 'Embarked'], drop_first=True)

        # Define features (X) and target (y)
        self.X = self.data.drop('Survived', axis=1)
        self.y = self.data['Survived']
        
        return self.X, self.y

# Step 2: Build and Train the RNN Model
class TitanicRNNModel:
    def __init__(self):
        self.model = None
        self.scaler = StandardScaler()

    def split_data(self, X, y, test_size=0.2, random_state=42):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        return X_train, X_test, y_train, y_test

    def build_model(self, input_shape):
        self.model = Sequential([
            SimpleRNN(16, activation='relu', input_shape=input_shape),
            Dense(8, activation='relu'),
            Dense(1, activation='sigmoid')  # Output layer for binary classification
        ])
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    def train_model(self, X_train, y_train, epochs=20, batch_size=32):
        X_train_scaled = self.scaler.fit_transform(X_train)
        # Reshape data for RNN (samples, timesteps, features)
        X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
        self.model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

    def evaluate_model(self, X_test, y_test):
        X_test_scaled = self.scaler.transform(X_test)
        X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
        loss, accuracy = self.model.evaluate(X_test_scaled, y_test, verbose=0)
        return accuracy

# Step 3: Save the Model
class ModelSaver:
    def __init__(self, model, scaler, feature_names):
        self.model = model
        self.scaler = scaler
        self.feature_names = feature_names

    def save_model(self, model_path, scaler_path, feature_names_path):
        self.model.save(model_path)  # Save TensorFlow model
        with open(scaler_path, 'wb') as scaler_file:
            pickle.dump(self.scaler, scaler_file)
        with open(feature_names_path, 'wb') as feature_file:
            pickle.dump(self.feature_names, feature_file)

# Step 4: Main Function
if __name__ == "__main__":
    # Load and preprocess data
    titanic_data = TitanicData('titanic.csv')
    titanic_data.load_data()
    X, y = titanic_data.preprocess_data()

    # Train and evaluate the RNN model
    titanic_rnn_model = TitanicRNNModel()
    X_train, X_test, y_train, y_test = titanic_rnn_model.split_data(X, y)
    titanic_rnn_model.build_model((X_train.shape[1], 1))
    titanic_rnn_model.train_model(X_train, y_train)
    accuracy = titanic_rnn_model.evaluate_model(X_test, y_test)
    print(f"RNN Model Accuracy: {accuracy * 100:.2f}%")

    # Save the model, scaler, and feature names
    model_saver = ModelSaver(titanic_rnn_model.model, titanic_rnn_model.scaler, X.columns.tolist())
    model_saver.save_model('titanic_rnn_model', 'scaler.pkl', 'feature_names.pkl')
